In [ ]:
from rapidfuzz import process, fuzz
import pandas as pd

In [ ]:
df_emd = pd.read_excel("c:\\Users\\MICILMEDS\\Downloads\\UPDATED STOCK LIST .xlsx")
df_mcl  = pd.read_excel("c:\\Users\\MICILMEDS\\Downloads\\20250925031502.xlsx")

In [ ]:
df_emd

In [ ]:
import pandas as pd
import re, time
from rapidfuzz import process, fuzz
from tqdm import tqdm

# Assuming df_emd and df_mcl are already loaded
# df_emd = pd.read_csv('your_emd_file.csv')
# df_mcl = pd.read_csv('your_mcl_file.csv')

THRESHOLD = 90
BLOCK_LEN = 2  # first N chars as blocking key

# --- STEP 1: CLEAN TEXT ---
def clean_text(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r'[^a-z0-9 ]', ' ', s)  # Fixed: added space after comma
    s = re.sub(r'\s+', ' ', s).strip()
    return s

# Apply cleaning - make sure column names exist
df_emd['clean_name'] = df_emd['Product Name'].map(clean_text)
df_mcl['clean_name'] = df_mcl['Product Name'].map(clean_text)

# Create blocking keys
df_emd['block'] = df_emd['clean_name'].str[:BLOCK_LEN]
df_mcl['block'] = df_mcl['clean_name'].str[:BLOCK_LEN]

# --- STEP 2: BLOCK-WISE MATCHING ---
start = time.time()
results = []

# Only keep blocks present in both datasets
blocks = sorted(set(df_emd['block']) & set(df_mcl['block']))
print(f"Processing {len(blocks)} blocks...")

for blk in tqdm(blocks, desc="Matching blocks"):
    emd_block = df_emd[df_emd['block'] == blk].reset_index(drop=True)
    mcl_block = df_mcl[df_mcl['block'] == blk].reset_index(drop=True)
    
    # Skip empty blocks
    if len(emd_block) == 0 or len(mcl_block) == 0:
        continue
    
    choices = mcl_block['clean_name'].tolist()
    
    # Better approach: Create mapping without using index
    # This handles duplicates by keeping all occurrences
    mcl_lookup = {}
    for _, mcl_row in mcl_block.iterrows():
        clean_name = mcl_row['clean_name']
        if clean_name not in mcl_lookup:
            mcl_lookup[clean_name] = {
                'Product Name': mcl_row['Product Name'],
                'Product Code': mcl_row['Product Code']
            }
    
    for _, row in emd_block.iterrows():
        try:
            match = process.extractOne(
                row['clean_name'],
                choices,
                scorer=fuzz.token_sort_ratio,
                score_cutoff=THRESHOLD
            )
            if match:
                matched_name, score, _ = match  # unpack 3 values
                matched_info = mcl_lookup[matched_name]
                
                results.append({
                    'Product Name': row['Product Name'],
                    'matched_name': matched_info['Product Name'],
                    'match_score': score
                })
        except Exception as e:
            print(f"Error processing row: {row['Product Name']} - {e}")
            continue

# Create results DataFrame
matches_df = pd.DataFrame(results)
print(f"✅ Completed in {time.time()-start:.2f} seconds for {len(matches_df)} matches")

# Optional: Display sample results
if len(matches_df) > 0:
    print("\nSample matches:")
    print(matches_df.head())
    print(f"\nMatch score distribution:")
    print(matches_df['match_score'].describe())
else:
    print("No matches found. Consider lowering the threshold or checking your data.")

# Optional: Save results
# matches_df.to_csv('fuzzy_matches.csv', index=False)

In [ ]:
matches_df

In [ ]:
matches_df.drop_duplicates(inplace=True)

In [ ]:
matches_df.reset_index(drop=True, inplace=True)

In [ ]:
matches_df.info()

In [ ]:
matches_df['product_code'].unique()

In [ ]:
matches_df['matched_code'].unique()

In [ ]:
df = pd.read_excel("c:\\Users\\MICILMEDS\\Documents\\Medi_final\\correct_data\\Category\\matched_Products.xlsx")

In [ ]:
matches_df.head()

In [ ]:
df.head()

In [ ]:
df_Merged = pd.merge(df, matches_df, how='inner', left_on='product_code', right_on='product_code')

In [ ]:
df_Merged.drop_duplicates(inplace=True)

In [ ]:
df_Merged[['product_code','image_path','matched_code']]

In [ ]:
import os
import shutil
import pandas as pd

# --- Assuming df_Merged is already loaded ---
# For testing, take only first 50 rows

# Base paths
base_path = r"C:\Users\MICILMEDS\Downloads\EMed_Image\html\uploads\product\main"
destination = r"C:\Users\MICILMEDS\Documents\Medi_final\correct_data\Category\Drive_images"

# Create destination folder if not exists
os.makedirs(destination, exist_ok=True)

for idx, row in df_Merged.iterrows():
    image_path = row['image_path']          # e.g. medicine/1_2.jpg
    matched_code = row['matched_code']      # e.g. MIC162999

    # Full source path
    src_path = os.path.join(base_path, image_path)

    # Extract suffix after the first underscore
    file_name = os.path.basename(image_path)   # e.g. 1_2.jpg
    suffix = file_name.split("_", 1)[-1]       # e.g. 2.jpg

    # Create new name
    new_name = f"{matched_code}_{suffix}"
    dest_path = os.path.join(destination, new_name)

    # Copy the image
    try:
        shutil.copy2(src_path, dest_path)
        print(f"✅ Copied: {src_path} --> {dest_path}")
    except FileNotFoundError:
        print(f"❌ Missing file: {src_path}")
    except Exception as e:
        print(f"⚠️ Error copying {src_path}: {e}")


In [ ]:
from PIL import Image, ImageDraw, ImageFont, UnidentifiedImageError
from pathlib import Path

folder = Path(r"C:\Users\MICILMEDS\Documents\Medi_final\correct_data\Category\Drive_images")

watermark_text = "MICYLMEDS"
font_size = 40
opacity = 60  # 0-255
angle = 30

# Load font
try:
    font = ImageFont.truetype("arial.ttf", font_size)
except:
    font = ImageFont.load_default()

for file in folder.glob("*.*"):
    if file.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
        continue

    try:
        with Image.open(file).convert("RGBA") as base:

            # Resize image; if fails, slightly adjust size
            try:
                base = base.resize((1024, 871))
            except Exception:
                # Fallback: scale proportionally if exact resize fails
                base.thumbnail((1024, 871))

            # Create watermark layer
            watermark_layer = Image.new("RGBA", base.size, (0, 0, 0, 0))
            draw = ImageDraw.Draw(watermark_layer)

            # Get text size
            bbox = draw.textbbox((0, 0), watermark_text, font=font)
            text_width = bbox[2] - bbox[0]
            text_height = bbox[3] - bbox[1]

            # Repeat watermark across image
            x_repeat = range(0, base.width, text_width + 100)
            y_repeat = range(0, base.height, text_height + 100)

            for x in x_repeat:
                for y in y_repeat:
                    draw.text((x, y), watermark_text, font=font, fill=(255, 255, 255, opacity))

            # Rotate watermark around center
            watermark_layer = watermark_layer.rotate(angle, expand=True)
            cx, cy = watermark_layer.width // 2, watermark_layer.height // 2
            bx, by = base.width // 2, base.height // 2
            watermark_layer = watermark_layer.crop((cx - bx, cy - by, cx - bx + base.width, cy - by + base.height))

            # Merge watermark with base image
            final = Image.alpha_composite(base, watermark_layer)

            # Save final image
            final.convert("RGB").save(file, quality=95)
            print(f"✅ Processed: {file.name}")

    except UnidentifiedImageError:
        print(f"⚠️ Skipped (not a valid image): {file.name}")
    except Exception as e:
        print(f"⚠️ Skipped ({e}): {file.name}")


In [ ]:
from pathlib import Path
import shutil
import math
from tqdm import tqdm  # pip install tqdm

# Source folder
source_folder = Path(r"C:\Users\MICILMEDS\Documents\Medi_final\correct_data\Category\Drive_images")
if not source_folder.exists():
    raise FileNotFoundError(f"Source folder does not exist: {source_folder}")

# Destination parent folder
dest_parent = Path(r"C:\Users\MICILMEDS\Documents\Medi_final\correct_data\Category\Drive_splitted")
dest_parent.mkdir(parents=True, exist_ok=True)

chunk_size = 1999

# Separate valid and empty files
all_files = [f for f in source_folder.iterdir() if f.is_file()]
valid_files = [f for f in all_files if f.stat().st_size > 0]
empty_files = [f for f in all_files if f.stat().st_size == 0]

# Remove empty files with progress bar
removed_files = []
for f in tqdm(empty_files, desc="Removing empty files", unit="file"):
    f.unlink()
    removed_files.append(f.name)

# Split valid files into batches
num_batches = math.ceil(len(valid_files) / chunk_size)
copied_files = []

for i in range(num_batches):
    batch_folder = dest_parent / f"batch_{i+1:02d}"
    batch_folder.mkdir(exist_ok=True)

    start_idx = i * chunk_size
    end_idx = start_idx + chunk_size
    batch_files = valid_files[start_idx:end_idx]

    for file in tqdm(batch_files, desc=f"Copying to {batch_folder.name}", unit="file"):
        # Keep original file name
        shutil.copy(file, batch_folder)
        copied_files.append(file.name)

# Summary
print("\n✅ Process Complete")
print(f"Total empty files removed: {len(removed_files)}")
if removed_files:
    print(f"Removed files: {removed_files}")

print(f"\nTotal valid files copied: {len(copied_files)}")
if copied_files:
    print(f"Copied files: {copied_files}")


In [ ]:
import pandas as pd
import os
import glob

folder_path = "c:\\Users\\MICILMEDS\\Documents\\Medi_final\\correct_data\\Category\\All Brand\\Data"
extract_files = glob.glob(os.path.join(folder_path,"*xlsx"))
df = pd.concat([pd.read_excel(file) for file in extract_files],ignore_index=True)


In [ ]:
df_allbrand = pd.read_excel("c:\\Users\\MICILMEDS\\Documents\\Medi_final\\correct_data\\Category\\All Brand\\All_brands.xlsx")

In [ ]:
df_allbrand.shape

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

In [ ]:
df[df['Product Name'].str.contains("", case=False, na=False)]


In [ ]:
total_sum = 0
for i in range(ord('A'),ord('Z')+1):
    letter = chr(i)
    data = df['Product Name'].str.lower().str.startswith(letter.lower()).sum()
    print(f"total product of {letter} are {data}")
    total_sum+=data
print(f"Total Data {total_sum}")

In [ ]:
df.head()

In [ ]:
df_new = pd.read_excel("c:\\Users\\MICILMEDS\\Downloads\\20250930044502.xlsx")
df_new.head()

In [ ]:
df_new=df_new.drop(['Key','key'],axis=1)

In [ ]:
df_anti = df[~df['Product Name'].isin(df_new['Product Name'])]

In [ ]:
df_anti.shape

In [ ]:
df_anti.reset_index(inplace=True)

In [ ]:
df[df['Product Name'].str.lower().str.contains("ciplox")]

In [ ]:
df_anti

In [ ]:
df_anti=df_anti.drop('index',axis=1)

In [ ]:
df_anti.to_excel("C:\\Users\\MICILMEDS\\Documents\\Medi_final\\correct_data\\Category\\remaining_data.xlsx",index=False)